In [1]:
epsilon = 0.1 # Параметр эпсилон при использовании эпсилон жадной стратегии
gamma = 0.8 # Коэффциент дисконтирования гамма
random_seed = 3 #Random seed
time_delay = 1 # Задержка времени при отрисовке процесса игры после обучения (секунды)
lr_rate = 0.9 #Коэффициент скорости обучения альфа

In [2]:
import gym
import numpy as np
import time
from IPython.display import clear_output


def generate_random_map(size, p, sd):
    """Generates a random valid map (one that has a path from start to goal)
    :param size: size of each side of the grid
    :param p: probability that a tile is frozen
    """
    valid = False
    np.random.seed(sd)

    # DFS to check that it's a valid path.
    def is_valid(res):
        frontier, discovered = [], set()
        frontier.append((0,0))
        while frontier:
            r, c = frontier.pop()
            if not (r,c) in discovered:
                discovered.add((r,c))
                directions = [(1, 0), (0, 1), (-1, 0), (0, -1)]
                for x, y in directions:
                    r_new = r + x
                    c_new = c + y
                    if r_new < 0 or r_new >= size or c_new < 0 or c_new >= size:
                        continue
                    if res[r_new][c_new] == 'G':
                        return True
                    if (res[r_new][c_new] not in '#H'):
                        frontier.append((r_new, c_new))
        return False

    while not valid:
        p = min(1, p)
        res = np.random.choice(['F', 'H'], (size, size), p=[p, 1-p])
        res[0][0] = 'S'
        res[-1][-1] = 'G'
        valid = is_valid(res)
    return ["".join(x) for x in res]

#Генерация карты
random_map = generate_random_map(size=6, p=0.8, sd = random_seed) #Создаем свою карту
env = gym.make("FrozenLake-v0", desc=random_map, is_slippery=False) #Инициализируем среду
print("Ваша карта")
env.render() #Выводим карту на экран

Ваша карта

SFFHFF
FHFFHF
FFFHHF
HFFHHF
FFFFFF
FFFFFG


In [3]:
def choose_action(state):
    action=0
    if np.random.uniform(0, 1) < epsilon:
        action = np.random.randint(0,env.action_space.n) #***
    else:
        action = np.random.choice(np.array(np.argwhere(Q[state, :] == np.amax(Q[state, :])).flatten().tolist()))
    return action

def learn(state, state2, reward, action, action2, done):
    if done:
        Q[state, action] = Q[state, action] + lr_rate * (reward - Q[state, action])
    else:
        Q[state, action] = Q[state, action] + lr_rate * (reward + gamma * Q[state2, action2]- Q[state, action])

In [ ]:
wins = 0
wins_in_row = 0
id_min = 0

from tqdm import tqdm
np.random.seed(random_seed)
total_games = 40000
max_steps = 100
Q = np.zeros((env.observation_space.n, env.action_space.n))
#Main cycle
for game in tqdm(range(total_games)):
    state = env.reset()
    t = 0
    action = #Выбор действия в самом начале каждой игры
    while t < max_steps:
              
        t += 1

        state2, reward, done, info = env.step(action)

        action2 =  #выбор действия как для следующего шага игры, так и для обновления ценности совершенного действия

        if t == max_steps:
          done = True  

        learn(state, state2, reward, action, action2, done) # action2 также передается в функцию обучения

        state = state2

        action = action2

        if done:
          break
        
    if state == 35:
        wins+=1
        wins_in_row +=1
        if id_min == 0 and wins_in_row == 5:
            id_min = game + 1
        
    else:
        wins_in_row = 0

In [ ]:
print("Количество побед в серии из 10 000 игр: ", wins)
print("Пять побед подряд впервые было одержано в игре ", id_min)